In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [4]:
from typing import Optional, List, Dict, Tuple

In [6]:
from configs.config import cfg, get_cfg_defaults
from core.datasets.dataset_loading_utils import load_dataset
from core.datasets.vq_dataset import DATALoader
from utils.vis_utils import plot_3d_global
from core.models.conv_vqvae import ConvVQMotionModel
from core.models.conformer_vqvae import ConformerVQMotionModel
import utils.rotation_conversions as geometry

/srv/share2/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
cfg = get_cfg_defaults()
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_768_1024_affine_varlen/conformer_768_1024_affine_varlen.yaml")

In [8]:
cfg.dataset.dataset_root

'/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/'

In [9]:
vqvae_model = ConformerVQMotionModel(
            cfg.vqvae,
        )

Sync is turned on False


In [19]:
vqvae_model.load("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_768_1024_affine_varlen/vqvae_motion.pt")

## Dataloader

In [10]:
render_ds, sampler_val, weights_val = load_dataset(
                ["t2m"],cfg, "test"
            )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6221/6221 [01:04<00:00, 95.95it/s]

Total number of motions 5329


In [21]:
render_dl = DATALoader(
    
            render_ds, batch_size=32, shuffle=False
        )

In [22]:
batch = (next(iter(render_dl)))
    
print(batch["names"])

['00_000000']


In [6]:
from utils.vis_utils.render_final import Renderer
renderer = Renderer("cuda")

In [7]:
from pathlib import Path


In [79]:
aa = model.motionEncoder( torch.randn((1, 80 , 271)).cuda() , True)

In [92]:
embed_motion_features = aa.mean() + torch.randn((1, 20 , 768)).to(model.device) * aa.var()

In [93]:
quantized_enc_motion, indices, commit_loss = model.vq(embed_motion_features)

In [94]:
decoded_motion_features = model.motionDecoder(quantized_enc_motion, True)

In [22]:
render_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/renders/"

In [40]:
data  = np.load('/coc/scratch/sanisetty3/music_motion/MOYO/new_joints_smpl/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-b_stageii.npy')

In [41]:
data.shape

(261, 135)

In [42]:
# rots = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(torch.Tensor(data[:,3:69].reshape(-1,22,3)))).reshape(-1,132)

In [60]:
# data  = np.load('/coc/scratch/sanisetty3/music_motion/MOYO/new_joints_smpl/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.npy')
data  = np.load('/coc/scratch/sanisetty3/music_motion/GTA/new_joints_smpl/000001.npy')
print(data.shape)
renderer.render(
                    motion_vec= torch.Tensor(data),
                    outdir=render_path,
                    step=0,
                    name="rnd2",
                )

(101, 135)


In [17]:
pwd

'/coc/scratch/sanisetty3/music_motion/TGM3D'

## Trans eval

In [22]:
from utils.motion_processing.hml_process import HMLProcess
from utils.motion_processing.aist_process import AISTProcess
from utils.eval_trans import calculate_R_precision, calculate_activation_statistics, calculate_diversity, calculate_frechet_distance
from core.datasets import dataset_TM_eval
from core.models.evaluator_wrapper import EvaluatorModelWrapper
from utils.word_vectorizer import WordVectorizer


In [23]:
hml_processor = HMLProcess(device)

In [24]:
aist_processor = AISTProcess(device)

In [25]:
vqvae_model = vqvae_model.eval().to(device)

In [76]:
w_vectorizer = WordVectorizer(
    "/srv/hays-lab/scratch/sanisetty3/music_motion/T2M-GPT/glove", "our_vab"
)
eval_wrapper = EvaluatorModelWrapper(cfg.eval_model)
tm_eval = dataset_TM_eval.DATALoader(
    1,
    w_vectorizer,
    unit_length=4,
)

Loading Evaluation Model Wrapper (Epoch 28) Completed!!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4384/4384 [00:06<00:00, 711.71it/s]

4248 4248
Pointer Pointing at 0


In [77]:
nb_sample = 0

mean_gpt = np.load(
    "/srv/hays-lab/scratch/sanisetty3/music_motion/T2M-GPT/checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/mean.npy"
)
std_gpt = np.load(
    "/srv/hays-lab/scratch/sanisetty3/music_motion/T2M-GPT/checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/std.npy"
)

motion_annotation_list = []
motion_pred_list = []

R_precision_real = 0
R_precision = 0

nb_sample = 0
matching_score_real = 0
matching_score_pred = 0

In [28]:
dir = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/new_joint_vecs/"

In [78]:
for batch in tqdm(tm_eval, position=0, leave=True):
    (
        word_embeddings,
        pos_one_hots,
        caption,
        sent_len,
        hml_motion,
        m_length,
        token,
        name,
    ) = batch
    break

  0%|                                                                                                                             | 0/4248 [00:00<?, ?it/s]


In [108]:
smpl_mean = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/Mean.npy")
smpl_std = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/Std.npy")

In [79]:
m_length[0]

tensor(60)

In [116]:
motion = hml_motion.squeeze().to(float)[:m_length[0]]
denorm = tm_eval.dataset.inv_transform(motion.detach().cpu())
denorm2  = (denorm - mean_gpt) / std_gpt

In [87]:

if "M" in name[0]:
    smpl = np.load(os.path.join(dir , f"180_{n[1:]}.npy"))
    smpl_motions = (aist_processor.aist6d_to_hml(smpl[:m_length[0]+1 , :135]))
else:
    smpl = np.load(os.path.join(dir , f"00_{n}.npy"))
    smpl_motions = (aist_processor.aist6d_to_hml(smpl[:m_length[0] +1, :135]))


In [124]:
smpl_denorm =  ((smpl_motions - smpl_motions.mean(-1 , keepdims = True))/smpl_motions.std(-1, keepdims = True)) * denorm.std(-1, keepdims = True).numpy() + denorm.mean(-1, keepdims = True).numpy() 

In [ ]:
smpl_motions-smpl_mean

In [115]:
# ((denorm[0] - denorm[0].mean())/denorm[0].std())
# denorm[0]

In [114]:
# ((smpl_motions[0] - smpl_motions[0].mean())/smpl_motions[0].std()) * denorm[0].std().numpy() + denorm[0].mean().numpy() 

In [19]:

# motion = motion.to(device)
et, em = eval_wrapper.get_co_embeddings(
    word_embeddings, pos_one_hots, sent_len, denorm, m_length
)


In [22]:
gt_mu, gt_cov = calculate_activation_statistics(em.cpu().numpy())

In [23]:
hml_data = []
pred_pose_eval = torch.zeros((bs, seq, motion.shape[-1])).cuda()
for i in range(len(smpl_motions)):
    motion_aa = hml_processor.to_aist_axis_angle_from6d(torch.Tensor(smpl_motions[i][:,:135]))
    pose_seq, body = hml_processor.aa2joints(motion_aa)
    data_hml, rec_ric_data = hml_processor.get_hml_rep(pose_seq[:,:22].cpu().numpy())
    pred_denorm = (data_hml - mean_gpt) / std_gpt
    pred_pose_eval[i : i + 1, : pred_denorm.shape[0], :] = torch.Tensor(pred_denorm)
    # hml_data.append(pred_denorm)


In [25]:
pred_pose_eval.shape

torch.Size([32, 196, 263])

In [26]:
et_pred, em_pred = eval_wrapper.get_co_embeddings(
    word_embeddings, pos_one_hots, sent_len, pred_pose_eval, m_length
)

In [31]:
mu, cov = calculate_activation_statistics(em_pred[:27].cpu().numpy())

In [32]:
mu.shape

(512,)

In [33]:
fid = calculate_frechet_distance(gt_mu, gt_cov, mu, cov)

In [34]:
fid

9.547529978650815

In [ ]:
bs, seq = motion.shape[0], motion.shape[1]

pred_pose_eval = torch.zeros((bs, seq, motion.shape[-1])).cuda()

for i in range(bs):
    pred_pose, ind, loss_commit = net(motion[i : i + 1, : m_length[i]])
    pred_denorm = val_loader.dataset.inv_transform(pred_pose.detach().cpu())
    pred_denorm = (pred_denorm - mean_gpt) / std_gpt

    pred_pose_eval[i : i + 1, : m_length[i], :] = pred_denorm

et_pred, em_pred = eval_wrapper.get_co_embeddings(
    word_embeddings, pos_one_hots, sent_len, pred_pose_eval, m_length
)

In [ ]:

for batch in tqdm(val_loader, position=0, leave=True):
    (
        word_embeddings,
        pos_one_hots,
        caption,
        sent_len,
        motion,
        m_length,
        token,
        name,
    ) = batch
    motion = motion.to(torch.float32)
    denorm = val_loader.dataset.inv_transform(motion.detach().cpu())
    denorm = (denorm - mean_gpt) / std_gpt

    motion = motion.cuda()
    et, em = eval_wrapper.get_co_embeddings(
        word_embeddings, pos_one_hots, sent_len, denorm, m_length
    )
    bs, seq = motion.shape[0], motion.shape[1]

    num_joints = 21 if motion.shape[-1] == 251 else 22

    pred_pose_eval = torch.zeros((bs, seq, motion.shape[-1])).cuda()

    for i in range(bs):
        pred_pose, ind, loss_commit = net(motion[i : i + 1, : m_length[i]])
        pred_denorm = val_loader.dataset.inv_transform(pred_pose.detach().cpu())
        pred_denorm = (pred_denorm - mean_gpt) / std_gpt

        pred_pose_eval[i : i + 1, : m_length[i], :] = pred_denorm

    et_pred, em_pred = eval_wrapper.get_co_embeddings(
        word_embeddings, pos_one_hots, sent_len, pred_pose_eval, m_length
    )

    motion_pred_list.append(em_pred)
    motion_annotation_list.append(em)

    temp_R, temp_match = calculate_R_precision(
        et.cpu().numpy(), em.cpu().numpy(), top_k=3, sum_all=True
    )
    R_precision_real += temp_R
    matching_score_real += temp_match
    temp_R, temp_match = calculate_R_precision(
        et_pred.cpu().numpy(), em_pred.cpu().numpy(), top_k=3, sum_all=True
    )
    R_precision += temp_R
    matching_score_pred += temp_match

    nb_sample += bs

motion_annotation_np = torch.cat(motion_annotation_list, dim=0).cpu().numpy()
motion_pred_np = torch.cat(motion_pred_list, dim=0).cpu().numpy()
gt_mu, gt_cov = calculate_activation_statistics(motion_annotation_np)
mu, cov = calculate_activation_statistics(motion_pred_np)

## New LOss

In [11]:
from core.models.smpl.body_model import BodyModel
import utils.rotation_conversions as geomtry

In [12]:
bm = BodyModel("/srv/hays-lab/scratch/sanisetty3/music_motion/motion-diffusion-model/body_models/smplh/neutral/model.npz")

In [13]:
for batch in render_dl:
    break

In [14]:
m6d = batch["motion"][:,:,:135]

In [15]:
m6d.shape

torch.Size([2, 80, 135])

In [16]:
bs , n , d = m6d.shape

In [76]:
root_trans = 

In [95]:
aa = geomtry.matrix_to_axis_angle(geomtry.rotation_6d_to_matrix(m6d[:,:,3:135].reshape(-1,22,6))).reshape(-1 , 66)

In [96]:
aa.shape

torch.Size([160, 66])

In [98]:
body = bm.forward(root_orient=aa[:,:3] , pose_body = aa[:,3:] , trans=m6d[:,:,:3].reshape(-1,3) , return_dict = True)

In [99]:
body.keys()

dict_keys(['v', 'f', 'Jtr', 'full_pose'])

In [103]:
body["Jtr"].reshape(bs , n , 52,3).shape

torch.Size([2, 80, 52, 3])

In [100]:
(body["Jtr"][1:] - body["Jtr"][:-1]).shape

torch.Size([159, 52, 3])

In [85]:
Loss = torch.nn.SmoothL1Loss()

In [86]:
Loss(body["Jtr"] , body["Jtr"])

tensor(0.)

In [113]:
m6d.shape

torch.Size([2, 80, 271])

In [20]:
def forward_new(motion_pred, motion_gt, mask=None):
    Loss = torch.nn.SmoothL1Loss()
    bs, n, d = motion_gt.shape  ## d = 135
    motion_pred_aa = geomtry.matrix_to_axis_angle(
        geomtry.rotation_6d_to_matrix(motion_pred[:, :, 3:135].reshape(-1, 22, 6))
    ).reshape(-1, 66)
    body_pred = bm.forward(
        root_orient=motion_pred_aa[:, :3],
        pose_body=motion_pred_aa[:, 3:],
        trans=motion_pred[:, :, :3].reshape(-1, 3),
        return_dict=True,
    )

    motion_gt_aa = geomtry.matrix_to_axis_angle(
        geomtry.rotation_6d_to_matrix(motion_gt[:, :, 3:135].reshape(-1, 22, 6))
    ).reshape(-1, 66)
    body_gt = bm.forward(
        root_orient=motion_gt_aa[:, :3],
        pose_body=motion_gt_aa[:, 3:],
        trans=motion_gt[:, :, :3].reshape(-1, 3),
        return_dict=True,
    )

    j_pred = body_pred["Jtr"].reshape(bs, n, 52, 3)[:, : 22, :]
    j_gt = body_gt["Jtr"].reshape(bs, n, 52, 3)[:, : 22, :]

    loss_pos = Loss(
        j_pred,
        j_gt,
    )
    loss_vel = Loss(
        (j_pred[:, 1:] - j_pred[:, :-1]), (j_gt[:, 1:] - j_gt[:, :-1])
    )

    return loss_pos, loss_vel

In [23]:
motion_pred, _ , _ = vqvae_model(m6d[:, :, :135])

In [24]:
motion_pred.shape

torch.Size([2, 80, 135])

In [25]:
forward_new(motion_pred ,m6d )

(tensor(0.1199, grad_fn=<SmoothL1LossBackward0>),
 tensor(0.0787, grad_fn=<SmoothL1LossBackward0>))

In [29]:
np.load("../HumanMotionSMPL/Mean.npy")[

array([ 2.42843907e-02,  1.13441283e+00, -8.98784186e-03, -2.40984987e-03,
       -6.52230864e-04, -9.92232620e-04, -6.47345805e-03,  9.72569679e-01,
        1.95284334e-02,  9.65837892e-01, -1.06665792e-01, -2.10577656e-02,
        1.12139878e-01,  8.77811686e-01,  2.35941030e-01,  9.67205206e-01,
        8.93590907e-02,  2.49379556e-02, -9.78139769e-02,  8.75245933e-01,
        2.57273114e-01,  9.84242926e-01, -1.86104024e-02,  8.78357464e-03,
        1.86940623e-02,  9.51855687e-01, -1.40547394e-01,  9.64632180e-01,
        6.87693139e-02, -4.39669236e-02, -6.46595745e-02,  7.50103124e-01,
       -4.13668740e-01,  9.68300201e-01, -5.46530094e-02,  8.35328890e-03,
        4.05742119e-02,  7.39468683e-01, -4.42177031e-01,  9.91846624e-01,
       -2.63669251e-03, -1.29083900e-02,  2.03791298e-03,  9.82947577e-01,
       -6.16558589e-02,  9.84936983e-01, -4.41092218e-04,  6.37313024e-02,
       -1.02082998e-02,  9.64401329e-01,  1.14246273e-01,  9.86655531e-01,
       -2.35986068e-02, -

In [29]:
pwd

'/coc/scratch/sanisetty3/music_motion/TGM3D'

In [31]:
cd PyMAF-X/

/coc/scratch/sanisetty3/PyMAF-X


In [ ]:
!python -m apps.demo_smplx --image_folder examples/coco_images --detection_threshold 0.3 --pretrained_model data/pretrained_model/PyMAF-X_model_checkpoint_v1.1.pt --misc TRAIN.BHF_MODE full_body MODEL.PyMAF.HAND_VIS_TH 0.1